In [1]:
%cd ~/research/split_pgl
%load_ext autoreload
%autoreload 2

from data.data import dataset_attributes, shift_types, prepare_data, log_data
from utils import set_seed, Logger, CSVBatchLogger, log_args, get_model, hinge_loss
from notebooks.example_args import MyCUBArgs
import torch 
from data import dro_dataset
import wandb, os
from train import train
from data.folds import Subset, ConcatDataset

mainargs = MyCUBArgs(wandb=False,
                     seed=0,
                     show_progress=False,
                     project_name='')  # default gpu = 0
part1_args = mainargs.part1_args
part2_args = mainargs.part2_args
args = part1_args
set_seed(args.seed)

train_data, val_data, test_data = prepare_data(args, train=True)


loader_kwargs = {
        "batch_size": args.batch_size,
        "num_workers": 0,
        "pin_memory": True,
    }

part1_loader = dro_dataset.get_loader(train_data,
                                      train=True,
                                      reweight_groups=None,
                                      **loader_kwargs)

criterion = torch.nn.CrossEntropyLoss(reduction="none")

model = torch.load('/home/thien/research/pseudogroups/CUB/splitpgl_sweep_logs/p0.9_wd0.0001_lr0.0001/part1_s0/best_model.pth')
model.to('cpu')
for batch in part1_loader:
    x, y, g, data_idx = batch 
    break 


/home/thien/research/split_pgl
Reading '/home/thien/research/datasets/cub/data/waterbird_complete95_forest2water2/metadata.csv'


In [2]:
output = model(x)
per_sample_loss = criterion(output, y)
n_groups = train_data.n_groups
group_vect = torch.arange(n_groups).unsqueeze(1)  # 4x1 column vect of group indicator
 # 4x32 indicator matrix such that each column corresponds to an example and the row entry corresponds to the group
group_map = (g==group_vect).float()

In [20]:
group_count  = group_map.sum(1)

In [21]:
test = []

tensor([21.,  5.,  0.,  6.])